In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py

In [2]:
def prefilter_items(data):
    # Уберем самые популярные товары (их и так купят)
    #popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique()
    #popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    
    #top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist()
    #data = data[~data['item_id'].isin(top_popular)]
    
    # Уберем самые НЕ популярные товары (их и так НЕ купят)
#    ##top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist()
#    ##data = data[~data['item_id'].isin(top_notpopular)]
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товары
    
    # ...
    print ('Модуль utils.py функция prefilter_items - полет нормальный')
    
def postfilter_items(user_id, recommednations):
    print ('Модуль utils.py функция postfilter_items - полет нормальный')

In [13]:
#print(sys.path)

In [14]:
# добавляем путь расположения файла в sys.path

sys.path.append('C:/Users/sklod/OneDrive/Рабочий стол/STUDY/!Теория/Q4.Рекомендательные системы/src')

In [15]:
import utils

In [16]:
utils.prefilter_items(0)

Модуль utils.py функция prefilter_items - полет нормальный


In [17]:
utils.postfilter_items(0,0)

Модуль utils.py функция postfilter_items - полет нормальный


3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [ ]:
def get_recommendations(user, model, sparse_user_item, N=5):
    """Рекомендуем топ-N товаров"""
    
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]],  # !!! 
                                    recalculate_user=True)]
    return res

In [19]:
def top_purchases(data):
    top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    top_purchases.sort_values('quantity', ascending=False, inplace=True)
    top_purchases = top_purchases[top_purchases['item_id'] != 999999]

In [20]:
def get_similar_item(item_id, N=2):
    """Находит товар, похожий на item_id"""
    recs = model.similar_items(itemid_to_id[item_id], N=2)  # N - количество товаров
    top_rec = recs[1][0]  
    return id_to_itemid[top_rec]

In [18]:
def get_similar_items_recommendation(user, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    top_users_purchases = top_purchases[top_purchases['user_id'] == user].head(N)
    res = top_users_purchases['item_id'].apply(lambda x: get_similar_item(x)).tolist()        
    return res

In [ ]:
def get_similar_users_recommendation(user, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    res = []
        
    # Находим топ-N похожих пользователей
    similar_users = model.similar_users(userid_to_id[user], N=N+1)
    similar_users = [rec[0] for rec in similar_users]
    similar_users = similar_users[1:]   # удалим юзера из запроса

    for user in similar_users:
        userid = id_to_userid[user] #own recommender works with user_ids
        res.extend(get_recommendations(userid, N=1))
    return res

In [21]:
import recommenders